In [1]:
import pandas as pd
import numpy as np
import re
import time
from urllib import parse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def file_to_df():
    df = pd.read_excel('apple_iphone_url.xlsx')
    df['inch'] = df['inch'].astype(str)
    df['germany_inch'] = df['inch'].apply(lambda x: re.sub("\.",",",x))
    df['russia_rom'] = df['rom'].apply(lambda x: re.sub("gb","",x))
    
    return df
    
def url_maker(df):
    df['us_url'] = "https://www.apple.com/shop/buy-iphone/"+df['family_model']+"/"+df['inch']+"-inch-display-"+df['rom']+"-"+df['color']+"-unlocked"
    df['uk_url'] = "https://www.apple.com/uk/shop/buy-iphone/"+df['family_model']+"/"+df['inch']+"-inch-display-"+df['rom']+"-"+df['color']
    df['china_url'] = "https://www.apple.com.cn/shop/buy-iphone/"+df['family_model']+"/"+df['china_code']+"/A"
    
    df['japan_url'] = 0
    df['germany_url'] = 0
    df['russia_url'] = 0

    for i in range(len(df)):
        df['japan_url'][i] = "https://www.apple.com/jp/shop/buy-iphone/"+df['family_model'][i]+"/"+df['inch'][i]+\
                    parse.quote(df['japan_code_inch'][i])+"-"+df['rom'][i]+"-"+parse.quote(df['japan_code_color'][i])
        df['germany_url'][i] = "https://www.apple.com/de/shop/buy-iphone/"+df['family_model'][i]+"/"+df['germany_inch'][i]+"%22-display-"+\
                               df['rom'][i]+"-"+parse.quote(df['germany_code'][i]) 

        if df['family_model'][i] == "iphone-se":
            df['russia_url'][i] = "https://www.apple.com/ru/shop/buy-iphone/"+df['family_model'][i]+"/"+df['germany_inch'][i]+"-"+\
                                    "%D0%B4%D1%8E%D0%B9%D0%BC%D0%BE%D0%B2%D1%8B%D0%B9-%D0%B4%D0%B8%D1%81%D0%BF%D0%BB%D0%B5%D0%B9-"+\
                                   df['russia_rom'][i]+"%D0%B3%D0%B1-"+parse.quote(df['russia_code'][i])
        else: 
             df['russia_url'][i] = "https://www.apple.com/ru/shop/buy-iphone/"+df['family_model'][i]+"/"+\
                                    "%D0%B4%D0%B8%D1%81%D0%BF%D0%BB%D0%B5%D0%B9-"+\
                                    df['germany_inch'][i]+"-"+\
                                    "-%D0%B4%D1%8E%D0%B9%D0%BC%D0%B0-"+\
                                   df['russia_rom'][i]+"%D0%B3%D0%B1-"+parse.quote(df['russia_code'][i])       

    df['india_url'] = "https://www.indiaistore.com/"+df['model']+"/"
    
    return df

In [8]:
def select_no(browser):
    no_answer = browser.find_element_by_class_name('as-inlinetradeup-selector')
    no_answer.click()
    print("click No")
    
def select_region(browser,post_code):
    try:
        input_region_code = browser.find_elements_by_xpath('//*[@id="postalCode"]')[0]
        input_region_code.send_keys(post_code)
        
        ok_button = browser.find_element_by_class_name("as-deliverydatesoverlay-geospan")
        ok_button.click()
        
        webdriver.ActionChains(browser).send_keys(Keys.ESCAPE).perform()
        
    except:
        browser.implicitly_wait(10)
        select_region(browser,post_code)
        
    print("Select region popup close")
          
def get_delivery_date(browser,post_code):
    #1. No 선택했는지 확인
    try:
        get_delivery = browser.find_element_by_xpath\
                            ('//*[@id="primary"]/summary-builder/div[2]/div[1]/materializer/div[2]/div/div/button')
    except:
        select_no(browser)
        browser.implicitly_wait(5)
        get_delivery = browser.find_element_by_xpath\
                            ('//*[@id="primary"]/summary-builder/div[2]/div[1]/materializer/div[2]/div/div/button')
       
    #get_delivery_yn = get_delivery.text
    
    #2. 지역명 선택되어있는지 확인
    if "Get delivery dates" in get_delivery_yn:
        browser.implicitly_wait(10)
        try:
            get_delivery.click()
        except :
            browser.execute_script("arguments[0].click();", get_delivery)

        browser.implicitly_wait(3)
        select_region(browser,post_code)
        print("start select region")
        
    browser.implicitly_wait(5)
    delivery_date = browser.find_element_by_class_name("as-purchaseinfo-dudeinfo-deliverymsg").text
    
    print("start find delivery date")
    if "Free" not in delivery_date:
        get_delivery_date(browser,post_code)
        
    print("get delievery date success: ",delivery_date )
    return delivery_date

def get_price_delivery(browser, url, country,i,post_code):
    browser.get(url)
    time.sleep(3)
    
    select_no(browser)
    delivery_date = get_delivery_date(browser,post_code)
    price = browser.find_element_by_class_name('form-choiceselectorlabel-twocolright').text
    
    price_delivery_list = []
    price_delivery_list.append([price, delivery_date])

    return price_delivery_list

In [4]:
df = file_to_df()
df = url_maker(df)

In [9]:
url_list =[('usa','us_url','10001'),('uk','uk_url','sw1a2aa'), ('china', 'china_url','100000'), 
           ('japan', 'japan_url',''), ('germany','germany_url','10115'), 
           ('russia','russia_url','101000'), ('india', 'india_url')]

browser = webdriver.Chrome()
time.sleep(5)

for url_col in url_list:
    country = url_col[0]
    res_col = url_col[0]+"res"
    col_nm = url_col[1]
    post_code = url_col[2]
    
    df[res_col] = 0
    for i in range(len(df['model'])):
        url = df[col_nm][i]
        price_delivery_list = get_price_delivery(browser, url,country,i,post_code)
        df[res_col][i] = price_delivery_list

click No


ElementClickInterceptedException: Message: element click intercepted: Element <div class="as-inlinetradeup-selector">...</div> is not clickable at point (692, 37). Other element would receive the click: <div class="localnav-price">...</div>
  (Session info: chrome=86.0.4240.198)


In [7]:
price_list

NameError: name 'price_list' is not defined

In [6]:
delivery_date = browser.find_element_by_class_name("as-purchaseinfo-dudeinfo-deliverymsg").text
delivery_date

'17 Nov - 19 Nov – Free'

In [10]:
df

,No,model,family_model,inch,rom,color,china_code,japan_code_inch,japan_code_color,germany_code,...,russia_rom,us_url,uk_url,china_url,japan_url,germany_url,russia_url,india_url,ukres,chinares
0,1,iphone-12,iphone-12,6.1,64gb,white,MGGN3CH,インチディスプレイ,ホワイト,weiß,...,64,https://www.apple.com/shop/buy-iphone/iphone-1...,https://www.apple.com/uk/shop/buy-iphone/iphon...,https://www.apple.com.cn/shop/buy-iphone/iphon...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,"[[From £29.12/mo.\nor £699**, 1-3 business days]]",0
1,2,iphone-12,iphone-12,6.1,64gb,black,MGGM3CH,インチディスプレイ,ブラック,schwarz,...,64,https://www.apple.com/shop/buy-iphone/iphone-1...,https://www.apple.com/uk/shop/buy-iphone/iphon...,https://www.apple.com.cn/shop/buy-iphone/iphon...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,"[[From £29.12/mo.\nor £699**, Mon 16 Nov – Free]]",0
2,3,iphone-12,iphone-12,6.1,64gb,blue,MGGQ3CH,インチディスプレイ,ブラック,blau,...,64,https://www.apple.com/shop/buy-iphone/iphone-1...,https://www.apple.com/uk/shop/buy-iphone/iphon...,https://www.apple.com.cn/shop/buy-iphone/iphon...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,"[[From £29.12/mo.\nor £699**, 17 Nov - 19 Nov ...",0
3,4,iphone-12,iphone-12,6.1,64gb,green,MGGT3CH,インチディスプレイ,グリーン,grün,...,64,https://www.apple.com/shop/buy-iphone/iphone-1...,https://www.apple.com/uk/shop/buy-iphone/iphon...,https://www.apple.com.cn/shop/buy-iphone/iphon...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,"[[From £29.12/mo.\nor £699**, Tue 17 Nov – Free]]",0
4,5,iphone-12,iphone-12,6.1,64gb,red,MGGP3CH,インチディスプレイ,レッド,rot,...,64,https://www.apple.com/shop/buy-iphone/iphone-1...,https://www.apple.com/uk/shop/buy-iphone/iphon...,https://www.apple.com.cn/shop/buy-iphone/iphon...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,"[[From £29.12/mo.\nor £699**, Mon 16 Nov – Free]]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,59,iphone-se,iphone-se,4.7,128gb,black,MHH43CH,インチのディスプレイ,ブラック,schwarz,...,128,https://www.apple.com/shop/buy-iphone/iphone-s...,https://www.apple.com/uk/shop/buy-iphone/iphon...,https://www.apple.com.cn/shop/buy-iphone/iphon...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-se/,"[[£18.70/mo. for 24 mo. at 0% APR**, Mon 16 No...",0
59,60,iphone-se,iphone-se,4.7,128gb,red,MHH63CH,インチのディスプレイ,レッド,rot,...,128,https://www.apple.com/shop/buy-iphone/iphone-s...,https://www.apple.com/uk/shop/buy-iphone/iphon...,https://www.apple.com.cn/shop/buy-iphone/iphon...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-se/,"[[£18.70/mo. for 24 mo. at 0% APR**, Mon 16 No...",0
60,61,iphone-se,iphone-se,4.7,256gb,white,MHH83CH,インチのディスプレイ,ホワイト,weiß,...,256,https://www.apple.com/shop/buy-iphone/iphone-s...,https://www.apple.com/uk/shop/buy-iphone/iphon...,https://www.apple.com.cn/shop/buy-iphone/iphon...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-se/,"[[£22.87/mo. for 24 mo. a

In [12]:
df.to_excel('usa_2차_11.12.xlsx')

In [11]:
df['us_res'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[[$1,099.00, Dec 4 - Dec 11 – Free]]    8
[[$979.00, Tomorrow – Free]]            5
[[$879.00, Tomorrow – Free]]            5
[[$879.00, Nov 23 - Nov 25 – Free]]     5
[[$729.00, Nov 23 - Nov 25 – Free]]     5
[[$779.00, Nov 23 - Nov 25 – Free]]     5
[[$1,199.00, Dec 4 - Dec 11 – Free]]    4
[[$829.00, Tomorrow – Free]]            4
[[$1,399.00, Dec 4 - Dec 11 – Free]]    4
[[$1,299.00, Dec 4 - Dec 11 – Free]]    4
[[$399.00, Tomorrow – Free]]            3
[[$999.00, Dec 4 - Dec 11 – Free]]      3
[[$549.00, Tomorrow – Free]]            3
[[$449.00, Tomorrow – Free]]            3
[[$999.00, ]]                           1
[[$829.00, ]]                           1
Name: us_res, dtype: int64